In [ ]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [ ]:
#|export
# Python native modules
import os
from torch.multiprocessing import Queue
from typing import Tuple,NamedTuple
# Third party libs
from fastcore.all import add_docs
import matplotlib.pyplot as plt
import torchdata.datapipes as dp
from IPython.core.display import clear_output
import torch
import numpy as np
# Local modules
from fastrl.core import Record
from fastrl.loggers.core import LoggerBase,LogCollector,is_record
# from fastrl.torch_core import *

In [ ]:
#|default_exp loggers.jupyter_visualizers

# Visualizers 
> Iterable pipes for displaying environments as they run using `typing.NamedTuples` with `image` fields

In [ ]:
#|export
class SimpleJupyterVideoPlayer(dp.iter.IterDataPipe):
    def __init__(self, 
                 source_datapipe=None, 
                 between_frame_wait_seconds:float=0.1
        ):
        self.source_datapipe = source_datapipe
        self.between_frame_wait_seconds = 0.1

    def dequeue(self): 
        while self.buffer: yield self.buffer.pop(0)

        
    def __iter__(self) -> Tuple[NamedTuple]:
        img = None
        for record in self.source_datapipe:
            # for o in self.dequeue():
            if is_record(record):
                if record.value is None: continue
                if img is None: img = plt.imshow(record.value)
                img.set_data(record.value) 
                plt.axis('off')
                display(plt.gcf())
                clear_output(wait=True)
            yield record
add_docs(
    SimpleJupyterVideoPlayer,
    """Displays video from a `source_datapipe` that produces `typing.NamedTuples` that contain an `image` field.
       This only can handle 1 env input.""",
    dequeue="Grabs records from the `main_queue` and attempts to display them"
)

In [ ]:
#|export
class ImageCollector(dp.iter.IterDataPipe):
    title:str='image'

    def __init__(self,source_datapipe):
        self.source_datapipe = source_datapipe

    def convert_np(self,o):
        if isinstance(o,torch.Tensor): return o.detach().numpy()
        elif isinstance(o,np.ndarray): return o
        else:                          raise ValueError(f'Expects Tensor or np.ndarray not {type(o)}')
    
    def __iter__(self):
        # for q in self.main_buffers: q.append(Record('image',None))
        yield Record(self.title,None)
        for steps in self.source_datapipe:
            if isinstance(steps,dp.DataChunk):
                for step in steps:
                    yield Record(self.title,self.convert_np(step.image))
            else:
                yield Record(self.title,self.convert_np(steps.image))
            yield steps

In [ ]:
from fastrl.envs.gym import GymDataPipe

In [ ]:
#|eval:false
%matplotlib inline

In [ ]:

pipe = GymDataPipe(['CartPole-v1'],None,n=100,seed=0,include_images=True)
pipe = ImageCollector(pipe)
pipe = SimpleJupyterVideoPlayer(pipe)

for o in pipe: pass

In [ ]:
#|hide
#|eval: false
!nbdev_export